## QHyper usecase

### Create instance of problem

In [1]:
from QHyper.problems import KnapsackProblem, TSPProblem

PROBLEM_TYPE = 'knapsack' # 'tsp'


# Each problem needs different parameters, because it depends on the number
# of variables and constraints
# Number of hyper_optimizer_bounds depends on the number of constraints,
# because each constraint requires one weights, and objective function also
# requires one

if PROBLEM_TYPE == 'knapsack':
    # Create knapsack with 3 items: 2 with weights 1 and
    # value 2 - (1, 2), and one with weight 1 and value 1 - (1, 1)
    problem = KnapsackProblem(max_weight=2, items=[(1, 2), (1, 2),(1, 1)])

    problem_config = {  # This is going to be used later
        'type': 'knapsack',
        'max_weight': 2,
        'items': [(1, 2), (1, 2),(1, 1)]
    }

    params_config = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2.5, 2.5],
    }
    hyper_optimizer_bounds = 3*[(1, 10)]
    penalty = 2

elif PROBLEM_TYPE == 'tsp':
    # Create Traveling Salesmam Problem with 3 cities
    problem = TSPProblem(number_of_cities=3)

    problem_config = {  # This is going to be used later
        'type': 'tsp',
        'number_of_cities': 3,
    }

    params_config = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2, 2, 2, 2],
    }
    hyper_optimizer_bounds = 5*[(1, 10)]
    penalty = -1


In [2]:
print(f"Variables used to describe objective function"
      f"and constraints: {problem.variables}")
print(f"Objective function: {problem.objective_function}")
print("Constraints:")
for constraint in problem.constraints:
    print(f"    {constraint}")


Variables used to describe objective functionand constraints: (x0, x1, x2, x3, x4)
Objective function: Polynomial(terms=defaultdict(<class 'float'>, {('x0',): -2.0, ('x1',): -2.0, ('x2',): -1.0}))
Constraints:
    Polynomial(terms=defaultdict(<class 'float'>, {('x3',): -1.0, ('x4',): -1.0, (): 1.0})) == Polynomial(terms=defaultdict(<class 'float'>, {}))
    Polynomial(terms=defaultdict(<class 'float'>, {('x0',): -1.0, ('x1',): -1.0, ('x2',): -1.0, ('x3',): 1.0, ('x4',): 2.0})) == Polynomial(terms=defaultdict(<class 'float'>, {}))


### Use VQA to solve knapsack problem

In [3]:
# Create a VQA instance with HQAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)
from QHyper.solvers import solver_from_config
from numpy import pi

# While we could use problem instace to create solver, the better approach
# is to use config, because it is easier to save it in JSON file and load it later

solver_config = {
    "solver": {
        "type": "vqa",
        "optimizer": {
            "type": "scipy",
            "maxfun": 200,
            "bounds": [(0, 2 * pi)] * 10
        },
        "pqc": {
            "type": "wfqaoa",
            "layers": 5,
            "limit_results": 20,
            "penalty": penalty,
            "backend": "default.qubit",
        },
        "params_inits": params_config
    },
    "problem": problem_config
}

vqa = solver_from_config(solver_config)

# # 2. Providing actual isntance of each class like VQA and Optimizer
# NOT RECOMMENDED

# from QHyper.solvers.vqa import VQA
# from QHyper.solvers.vqa.pqc import HQAOA
# from QHyper.optimizers import ScipyOptimizer

# vqa = VQA(problem, HQAOA(layers=5, penalty=penalty, backend='default.qubit'),
#           ScipyOptimizer(maxfun=200))


In [4]:
# Run VQA with provided initial parameters
solver_results = vqa.solve()

print("Solver results:")
print(f"Probabilities: {solver_results.probabilities}")
print(f"Best params: {solver_results.params}")


Solver results:
Probabilities: {'00000': 0.0732912838324004, '00001': 0.01812365507384847, '00010': 0.05407207652748644, '00011': 0.0701062245930107, '00100': 0.017498889261866067, '00101': 0.009292651366578255, '00110': 0.050293202934572685, '00111': 0.016844666311210565, '01000': 0.046624559969783416, '01001': 0.009582944060964814, '01010': 0.04374581517898236, '01011': 0.007336159588485337, '01100': 0.010700779894629564, '01101': 0.07661867874108277, '01110': 0.007507066502992263, '01111': 0.023008509073494634, '10000': 0.04662455996978336, '10001': 0.0095829440609648, '10010': 0.04374581517898232, '10011': 0.00733615958848532, '10100': 0.010700779894629532, '10101': 0.07661867874108272, '10110': 0.00750706650299228, '10111': 0.02300850907349465, '11000': 0.02873850917070903, '11001': 0.06522050516274285, '11010': 0.01700164098761874, '11011': 0.02196157538990109, '11100': 0.035843090675343484, '11101': 0.014437271029358385, '11110': 0.002494804987010662, '11111': 0.0545309266755007

In [5]:
from QHyper.util import (
    weighted_avg_evaluation, sort_solver_results, add_evaluation_to_results)

# Evaluate results with weighted average evaluation
print("Evaluation:")
print(weighted_avg_evaluation(
    solver_results.probabilities, problem.get_score,
    penalty=0, limit_results=10, normalize=True
))
print("Sort results:")
sorted_results = sort_solver_results(
    solver_results.probabilities, limit_results=10)

# Add evaluation to results
results_with_evaluation = add_evaluation_to_results(
    sorted_results, problem.get_score, penalty=penalty)

for result, (probability, evaluation) in results_with_evaluation.items():
    print(f"Result: {result}, "
          f"Prob: {probability:.5}, "
          f"Evaluation: {evaluation}")


Evaluation:
-1.2555153432454813
Sort results:
Result: 01101, Prob: 0.076619, Evaluation: -3
Result: 10101, Prob: 0.076619, Evaluation: -3
Result: 00000, Prob: 0.073291, Evaluation: 0
Result: 00011, Prob: 0.070106, Evaluation: 2
Result: 11001, Prob: 0.065221, Evaluation: -4
Result: 11111, Prob: 0.054531, Evaluation: 2
Result: 00010, Prob: 0.054072, Evaluation: 2
Result: 00110, Prob: 0.050293, Evaluation: -1
Result: 01000, Prob: 0.046625, Evaluation: 2
Result: 10000, Prob: 0.046625, Evaluation: 2


#### Using hyper optimizers

In [6]:
# Additionally other optimizer can be used to tune some parameters, in below
# example, Random optimzier will change weights (hyper_args) and choose ones
# that gives the best results after runnign 200 iteration of scipy minimizer

solver_config = {
    "solver": {
        "type": "vqa",
        "optimizer": {
            "type": "scipy",
            "maxfun": 200,
            "bounds": [(0, 2 * pi)] * 10
        },
        "pqc": {
            "type": "wfqaoa",
            "layers": 5,
            "limit_results": 20,
            "penalty": penalty,
        },
        "params_inits": params_config,
        "hyper_optimizer": {
            "type": "random",
            "processes": 5,
            "number_of_samples": 100,
            "bounds": hyper_optimizer_bounds,
        }
    },
    "problem": problem_config
}
vqa = solver_from_config(solver_config)


In [7]:
solver_results = vqa.solve()

print("Solver results:")
print(f"Probabilities: {solver_results.probabilities}")
print(f"Best params: {solver_results.params}")


Solver results:
Probabilities: {'00000': 0.036215643738056184, '00001': 0.009732461820058054, '00010': 0.01512752159177665, '00011': 0.004898970085671026, '00100': 0.009811070774507602, '00101': 0.009995230222961251, '00110': 0.17020709388664948, '00111': 0.009835147114974447, '01000': 0.009633808259437516, '01001': 6.551348341363583e-05, '01010': 0.06818393104282872, '01011': 0.0032310466245177418, '01100': 0.008494761842195329, '01101': 0.13505509463529042, '01110': 0.01678625146275098, '01111': 0.01106622875202749, '10000': 0.00963380825943751, '10001': 6.551348341363564e-05, '10010': 0.06818393104282863, '10011': 0.0032310466245177435, '10100': 0.00849476184219533, '10101': 0.13505509463529033, '10110': 0.01678625146275097, '10111': 0.011066228752027488, '11000': 0.018086704657015597, '11001': 0.14481572304362988, '11010': 0.005243829793381593, '11011': 0.013867749560177636, '11100': 0.009298811833917654, '11101': 0.0028672422311025393, '11110': 0.010112918453326875, '11111': 0.024

In [8]:
print("Evaluation:")
print(weighted_avg_evaluation(
    solver_results.probabilities, problem.get_score,
    penalty=0, limit_results=10, normalize=True
))
print("Sort results:")
sorted_results = sort_solver_results(
    solver_results.probabilities, limit_results=10)

results_with_evaluation = add_evaluation_to_results(
    sorted_results, problem.get_score, penalty=penalty)

for result, (probability, evaluation) in results_with_evaluation.items():
    print(f"Result: {result}, "
          f"Prob: {probability:.5}, "
          f"Evaluation: {evaluation}")


Evaluation:
-2.241798915063301
Sort results:
Result: 00110, Prob: 0.17021, Evaluation: -1
Result: 11001, Prob: 0.14482, Evaluation: -4
Result: 01101, Prob: 0.13506, Evaluation: -3
Result: 10101, Prob: 0.13506, Evaluation: -3
Result: 01010, Prob: 0.068184, Evaluation: -2
Result: 10010, Prob: 0.068184, Evaluation: -2
Result: 00000, Prob: 0.036216, Evaluation: 0
Result: 11111, Prob: 0.024851, Evaluation: 2
Result: 11000, Prob: 0.018087, Evaluation: 2
Result: 01110, Prob: 0.016786, Evaluation: 2
